# SETTINGS

## Libraries

In [13]:
import fitz
import os
import shutil
import re
import pandas as pd
import numpy as np
from functools import wraps
import time

def timeit(func):
    @wraps(func)
    def timeit_wrapper(*args, **kwargs):
        start_time = time.perf_counter()
        result = func(*args, **kwargs)
        end_time = time.perf_counter()
        total_time = end_time - start_time
        # print(f'Function {func.__name__}{args} {kwargs} Took {total_time:.4f} seconds')
        print(f'Function {func.__name__} Took {total_time:.4f} seconds')
        return result
    return timeit_wrapper

os.getcwd()

'C:\\Users\\juani\\Documents\\3_My_Jupiter_Notebooks\\5_Galicia\\7_boletines'

## Loading Files

In [14]:
#levantamos el documento
my_path = r'data/seccion_segunda_20230719.pdf'
doc = fitz.open(my_path)

In [15]:
# juntamos las páginas:
text = ''
for n,page in enumerate(doc):
    if n >1:
        text += page.get_text()

## Preprocessing

In [16]:
def doc_to_string(doc):
    '''
    def: convertir de formato doc (fitz) a un único string de texto para todo el documento levantado.
    input: doc (fitz)
    output: string
    '''    
    text = ''
    for n,page in enumerate(doc):
        if n >1:
            text += page.get_text()
    return text

text = doc_to_string(doc)

In [17]:
def prepro(text):
    ''' 
    Función de preprocesamiento.
    '''
    # 1. Eliminamos las cabeceras con fecha y numero de boletín:
    
    pattern = r'BOLETÍN OFICIAL Nº.*?de \d{4}'
    matches = re.findall(pattern, text, re.DOTALL)
    
    for match in matches:
        text = text.replace(match, '')
    
    # 2. Eliminamos los #F6917090F#
    pattern = r'#[IF]\d+I#|#[IF]\d+F#\n'
    text = re.sub(pattern, '', text)
    return text

text = prepro(text)

In [19]:
def split_by_chapter(text):
    ''' 
    def: particiona el texto en función de los títulos principales.
    
    '''
    headers = ['CONTRATOS SOBRE PERSONAS JURÍDICAS', 'CONVOCATORIAS Y AVISOS COMERCIALES', 'EDICTOS JUDICIALES']
    pattern = '|'.join(re.escape(header) for header in headers)
    matches = re.finditer(pattern, text)

    # Extract separators from matches
    separators = [match.group(0) for match in matches]

    # Split the text using the pattern
    chapters = re.split(pattern, text)

    # Remove empty strings from the result
    chapters = [chapter.strip() for chapter in chapters if chapter.strip()]

    # Label 'undefined' for the remaining separators
    separators.extend(['undefined'] * (len(chapters) - len(separators)))

    return chapters, separators

chapters,separators = split_by_chapter(text)
separators

['CONTRATOS SOBRE PERSONAS JURÍDICAS',
 'CONVOCATORIAS Y AVISOS COMERCIALES',
 'EDICTOS JUDICIALES',
 'CONVOCATORIAS Y AVISOS COMERCIALES',
 'EDICTOS JUDICIALES']

In [20]:
def split_by_e(chapters,chapter_name):
    '''
    def: particiona cada capítulo por escritura y genera un dataframe con nombre de capítulo // escritura // texto.
    '''
    separator_regex = r'e\.\s\d{2}/\d{2}/\d{4}\sN°\s\d+/\d+\sv\.\s\d{2}/\d{2}/\d{4}'
    sep_col = []
    ch_col = []
    part_col = []

    for n,chapter in enumerate(chapters):    
        separators = re.findall(separator_regex, chapter)
        parts = re.split(separator_regex, chapter)
        parts = [l for l in parts if bool(l)]
        if len(parts)==len(separators):
            sep_col.extend(separators)
            part_col.extend(parts)
            ch_n = [chapter_name[n] for m in range(len(parts))]
            ch_col.extend(ch_n)
    temp = pd.DataFrame({'chapter':ch_col,'escrito':sep_col,'texto':part_col})
    return temp   
temp = split_by_e(chapters,separators)
temp.head()

,chapter,escrito,texto
0,CONTRATOS SOBRE PERSONAS JURÍDICAS,e. 19/07/2023 N° 55783/23 v. 19/07/2023,SOCIEDADES ANÓNIMAS\nACEGO S.A.\nEscritura Nº ...
1,CONTRATOS SOBRE PERSONAS JURÍDICAS,e. 19/07/2023 N° 55773/23 v. 19/07/2023,\nAGROQUIM NORTE S.A.\nEsc 52 del 03/07/2023 F...
2,CONTRATOS SOBRE PERSONAS JURÍDICAS,e. 19/07/2023 N° 55779/23 v. 19/07/2023,\n\n\n \nALCARI S.A.\nNúmero Correlativo 1.707...
3,CONTRATOS SOBRE PERSONAS JURÍDICAS,e. 19/07/2023 N° 55528/23 v. 19/07/2023,\nB2FI S.A.\n1) 17/07/2023. 2) Francisco LEVIS...
4,CONTRATOS SOBRE PERSONAS JURÍDICAS,e. 19/07/2023 N° 55539/23 v. 19/07/2023,\nCMIX ARGENTINA S.A.\nCUIT: 30-71088679-9.Com...


In [21]:
def procesar(text):
    '''
    def: integra las funciones previas.
    input: texto (string)
    output: dataframe
    '''
    text = prepro(text)
    chapters,separators = split_by_chapter(text)
    temp = split_by_e(chapters,separators)
    return temp

temp = procesar(text)
temp.head()

,chapter,escrito,texto
0,CONTRATOS SOBRE PERSONAS JURÍDICAS,e. 19/07/2023 N° 55783/23 v. 19/07/2023,SOCIEDADES ANÓNIMAS\nACEGO S.A.\nEscritura Nº ...
1,CONTRATOS SOBRE PERSONAS JURÍDICAS,e. 19/07/2023 N° 55773/23 v. 19/07/2023,\nAGROQUIM NORTE S.A.\nEsc 52 del 03/07/2023 F...
2,CONTRATOS SOBRE PERSONAS JURÍDICAS,e. 19/07/2023 N° 55779/23 v. 19/07/2023,\n\n\n \nALCARI S.A.\nNúmero Correlativo 1.707...
3,CONTRATOS SOBRE PERSONAS JURÍDICAS,e. 19/07/2023 N° 55528/23 v. 19/07/2023,\nB2FI S.A.\n1) 17/07/2023. 2) Francisco LEVIS...
4,CONTRATOS SOBRE PERSONAS JURÍDICAS,e. 19/07/2023 N° 55539/23 v. 19/07/2023,\nCMIX ARGENTINA S.A.\nCUIT: 30-71088679-9.Com...


In [22]:
@timeit
def procesar_carpeta(folder,ls):
    
    if not ls:
        ls = os.listdir(folder)
        ls = [l for l in ls if '.pdf' in l]    
    df = None
    
    for file in ls:
        doc = fitz.open(my_path)
        text = doc_to_string(doc)
        temp = procesar(text)
        df = pd.concat([df,temp])
    
    
    df.reset_index(inplace=True)
    df.drop('index', axis=1, inplace=True)
    return df

---

# EJECUCION

In [23]:
folder = 'C:\\Users\\juani\\Documents\\3_My_Jupiter_Notebooks\\5_Galicia\\7_boletines'+'\\data\\'
boletines_ls = os.listdir(folder)
boletines_ls = [element for element in boletines_ls if '(' not in element]
boletines_ls = [element for element in boletines_ls if re.search(r'seccion_segunda_202', element)]
len(boletines_ls)

1320

In [152]:
df = procesar_carpeta(folder,boletines_ls)

Function procesar_carpeta Took 334.6420 seconds


In [153]:
df.to_pickle('df.pkl')

---

# trameos los documentos de 2017 a 2019

In [160]:
# 'C:\\Users\\juani\\Documents\\3_My_Jupiter_Notebooks\\5_Galicia\\7_boletines'
source_folder = r'D:\7_boletines\data\CABA'
destination_folder = r'C:\\Users\\juani\\Documents\\3_My_Jupiter_Notebooks\\5_Galicia\\7_boletines\\data'
boletines = os.listdir(source_folder)
boletines_repetidos = [element for element in boletines if '(' in element]
len(boletines_repetidos), len(boletines)
sel_bol = [bol for bol in boletines if bol not in boletines_repetidos]
sel_bol_17 = [element for element in sel_bol if re.search(r'seccion_segunda_2017', element)]
sel_bol_18 = [element for element in sel_bol if re.search(r'seccion_segunda_2018', element)]
sel_bol_19 = [element for element in sel_bol if re.search(r'seccion_segunda_2019', element)]
sel_1 = sel_bol_17+ sel_bol_18+sel_bol_19
len(sel_1)

736

In [162]:
# Paths for the source and destination folders

# Iterate through the file list and copy each file
for file_name in sel_1:
    source_path = os.path.join(source_folder, file_name)
    destination_path = os.path.join(destination_folder, file_name)
    
    shutil.copy2(source_path, destination_path)

print("Files copied successfully.")

Files copied successfully.


# Repetimos Ejecución

In [164]:
df2 = procesar_carpeta(folder,sel_1)

Function procesar_carpeta Took 200.9118 seconds


In [165]:
df2.to_pickle('df2.pkl')

In [171]:
df3 = pd.concat([df,df2])
df3.to_pickle('df3.pkl')

In [172]:
df3

,chapter,escrito,texto
0,CONTRATOS SOBRE PERSONAS JURÍDICAS,e. 19/07/2023 N° 55783/23 v. 19/07/2023,SOCIEDADES ANÓNIMAS\nACEGO S.A.\nEscritura Nº ...
1,CONTRATOS SOBRE PERSONAS JURÍDICAS,e. 19/07/2023 N° 55773/23 v. 19/07/2023,\nAGROQUIM NORTE S.A.\nEsc 52 del 03/07/2023 F...
2,CONTRATOS SOBRE PERSONAS JURÍDICAS,e. 19/07/2023 N° 55779/23 v. 19/07/2023,\n\n\n \nALCARI S.A.\nNúmero Correlativo 1.707...
3,CONTRATOS SOBRE PERSONAS JURÍDICAS,e. 19/07/2023 N° 55528/23 v. 19/07/2023,\nB2FI S.A.\n1) 17/07/2023. 2) Francisco LEVIS...
4,CONTRATOS SOBRE PERSONAS JURÍDICAS,e. 19/07/2023 N° 55539/23 v. 19/07/2023,\nCMIX ARGENTINA S.A.\nCUIT: 30-71088679-9.Com...
...,...,...,...
202395,EDICTOS JUDICIALES,e. 17/07/2023 N° 55072/23 v. 21/07/2023,\nJUZGADO NACIONAL EN LO COMERCIAL NRO. 26 - S...
202396,EDICTOS JUDICIALES,e. 18/07/2023 N° 50514/23 v. 19/07/2023,\nJUZGADO NACIONAL EN LO CIVIL NRO. 6 - SECRET...
202397,EDICTOS JUDICIALES,e. 17/07/2023 N° 54813/23 v. 21/07/2023,\nCÁMARA FEDERAL DE SEGURIDAD SOCIAL SECRETARÍ...
202398,EDICTOS JUDICIALES,e. 17/07/2023 N° 55045/23 v. 21/07/2023,\nREMATES JUDICIALES \nANTERIORES\nJUZGADO NAC...


# ESCRIBANOS

In [ ]:
Verónica Olicino (Mat. 5340)
María Beatriz Oscar Collins (Mat 252)
Guillermo Zoppi (Mat 4357)
María Martha Fermepin (Mat 3726)
María Beatriz Benchoa Echeverria (Mat -)
Cinthia Tomasini (Mat 4839)
María Marcela Coppola (Mat 5411)
Ricardo A. Doldan Aristizabal (Mat 4563)
Natalia S. Alonso (Mat 5261)
Guillermo A. Mendez (Mat 2410)

In [224]:
print(df3[df3['chapter']=='CONTRATOS SOBRE PERSONAS JURÍDICAS']['texto'].tolist()[12])


GC SOLUCIONES S.A.
Por Escritura Nº 819 F° 2158 Reg. 553 fecha 18/07/2023, CALABRESI, Mauricio Sebastián 11/11/1980, 28504814, 
60%, 180000 acciones/votos Martin Castro 2338 Ituzaingó Prov. Bs. As.; VILLARROEL, Juan Adrián, 20/08/1975, 
24856038, 20%, 60000 acciones/votos, Constantino Gaito 504 Lomas de Zamora, Prov. Bs. As., Accinti, Mariano 
Daniel 21/10/1982, 29620915, 20%, 20000 acciones/votos, Salcedo 3483, Castelar, Prov. Bs. As., todos argentinos, 
comerciantes, solteros. OBJETO: venta de materiales para la construcción, materiales para instalaciones eléctricas 
y de gas, como así también los servicios realizados por mantenimiento en obras en curso o edificios o viviendas 
en general. Las actividades que lo requieran serán ejercidas por profesionales con título habilitante. Cierre de 
Ejercicio 31/12. Capital $ 300.000, 300000 acciones ordinarias, nominativas de un (1) peso cada una se suscriben 
en un 100% y se integran en un 25%. Duración 30 años. Directorio por 3 ejercicios:

In [188]:
(df3['texto'].eq('Collins')).any()

False

---

# Multicolumna

In [236]:
def doc_to_string_multi(doc):
    '''
    def: convertir de formato doc (fitz) a un único string de texto para todo el documento levantado.
    input: doc (fitz)
    output: string
    '''    
    text = ''
    for n,page in enumerate(doc):
        text += page.get_text()
    return text

text = doc_to_string_multi(doc)

In [254]:
def split_by_chapter_multi(text):
    ''' 
    def: particiona el texto en función de los títulos principales.
    
    '''
    headers = ['Contratos sobre \nPersonas Jurídicas', 'Convocatorias y \nAvisos Comerciales', 'Edictos Judiciales','Partidos Políticos','Información y Cultura']
    pattern = '|'.join(re.escape(header) for header in headers)
    matches = re.finditer(pattern, text)

    # Extract separators from matches
    separators = [match.group(0) for match in matches]

    # Split the text using the pattern
    chapters = re.split(pattern, text)

    # Remove empty strings from the result
    chapters = [chapter.strip() for chapter in chapters if chapter.strip()]

    # Label 'undefined' for the remaining separators
    separators.extend(['undefined'] * (len(chapters) - len(separators)))

    return chapters, separators

chapters,separators = split_by_chapter_multi(text)
separators

['Edictos Judiciales',
 'Partidos Políticos',
 'Información y Cultura',
 'Edictos Judiciales',
 'Partidos Políticos',
 'Contratos sobre \nPersonas Jurídicas',
 'Convocatorias y \nAvisos Comerciales',
 'Edictos Judiciales',
 'Partidos Políticos',
 'Partidos Políticos',
 'Convocatorias y \nAvisos Comerciales',
 'Edictos Judiciales',
 'undefined']

In [10]:
print(chapters[6])

IndexError: list index out of range

In [12]:
print(chapters[0])

SOCIEDADES ANÓNIMAS
ACEGO S.A.
Escritura Nº 757 F° 2011 Reg. 553 de fecha 5/7/2023. Federico Gastón GOÑI, nacido el 21/5/1986, DNI 32.257.649, 
arquitecto, domiciliado en Calle 9, Nro 1521, Piso 2, Depto A, La Plata, Pcia Bs As, y Gabriel Sebastian GRAU, 
nacido el 17/6/1984, DNI 30.923.019, empresario, domiciliada en Calle 49 Nro 339, Planta Baja, Depto B, La Plata, 
Pcia Bs As; ambos argentinos y solteros. Objeto: A) Realización de obras públicas y/o privadas, por contratación 
directa, licitación pública y/o privada y cualquier sistema vigente y/o a crearse, ejecución de proyecto, dirección, 
administración, construcción, reparación y reformación de todo tipo de obras de ingeniería y arquitectura de 
inmuebles, incluyendo las actividades de la albañilería, herrería, electricidad y afines.- B) Compra, venta, 
importación, exportación, y comercialización de todo tipo de materiales, artículos e insumos para la construcción, 
incluyendo materiales de corralón, materiales eléctricos, maq

In [225]:
#levantamos el documento
my_path = r'data/seccion_segunda_20161228.pdf'
doc = fitz.open(my_path)
text = doc_to_string(doc)
temp = procesar(text)

In [243]:
chapters, headers = split_by_chapter(text)


In [242]:
headers

['undefined']

In [241]:
print(text)

1. Contratos sobre Personas Jurídicas
2. Convocatorias y Avisos Comerciales
Segunda Sección 
3. Edictos Judiciales
4. Partidos Políticos
5. Información y Cultura
Precio $ 20,00
Los documentos que aparecen en el BOLETÍN OFICIAL DE LA REPÚBLICA 
ARGENTINA serán tenidos por auténticos y obligatorios por el efecto de esta 
publicación y por comunicados y suficientemente circulados dentro de todo el 
territorio nacional (Decreto Nº 659/1947). La  edición electrónica del Boletín Oficial 
produce idénticos efectos jurídicos que su edición impresa (Decreto Nº 207/2016). 
PRESIDENCIA DE LA NACIÓN
SecretarÍa LegaL y tÉcnica: DR. PABLO CLUSELLAS - Secretario
DirecciÓn nacionaL DeL regiStro oficiaL: LIC. RICARDO SARINELLI - Director nacional
e-mail: dnro@boletinoficial.gob.ar
registro nacional de la Propiedad intelectual nº 5.218.874
DomiciLio LegaL: Suipacha 767-c1008aao - ciudad autónoma de Buenos aires
tel. y fax 5218-8400 y líneas rotativas
Segunda Sección
Buenos Aires,
miércoles 28 
de diciem

In [229]:
temp

,chapter,escrito,texto
0,undefined,e. 28/12/2016 N° 99392/16 v. 28/12/2016,Miércoles 28 de diciembre de 2016 \nSegunda Se...
1,undefined,e. 28/12/2016 N° 99288/16 v. 28/12/2016,\nISAGRO ARGENTINA LIMITADA S.R.L.\nPor Reunió...
2,undefined,e. 28/12/2016 N° 99320/16 v. 28/12/2016,\nJEETAN S.R.L.\nPor instrumento privado de 19...
3,undefined,e. 28/12/2016 N° 99327/16 v. 28/12/2016,\nLIBERCAM S.R.L.\nPor instrumento privado de ...
4,undefined,e. 28/12/2016 N° 99450/16 v. 28/12/2016,\nMAGRIN S.R.L.\nInstrumento \nprivado: \n26/1...
...,...,...,...
130,undefined,e. 27/12/2016 N° 98838/16 v. 02/01/2017,\nJUZGADO FEDERAL \nSECRETARÍA PENAL NRO. 3 \n...
131,undefined,e. 23/12/2016 N° 97712/16 v. 29/12/2016,\nJUZGADO FEDERAL EN LO CRIMINAL \nY CORRECCIO...
132,undefined,e. 22/12/2016 N° 97326/16 v. 28/12/2016,\nJUZGADO FEDERAL EN LO CRIMINAL \nY CORRECCIO...
133,undefined,e. 26/12/2016 N° 98540/16 v. 30/12/2016,\nJUZGADO FEDERAL EN LO CRIMINAL \nY CORRECCIO...


In [228]:
print(text)

 Miércoles 28 de diciembre de 2016 
Segunda Sección 
BOLETÍN OFICIAL Nº 33.532 
3
puesta en marcha de sistemas privados de 
Medicina Asistencial Prepagos y/o para Obras 
Sociales y/o Sistemas de Capacitación; Audi-
toria y contralor de prestaciones asistenciales; 
Relevamientos sanitarios; Arquitectura sanita-
ria; técnicas de productividad y Eficiencia de 
establecimientos sanitarios; Asesoramientos 
en equipamientos hospitalarios; Estudios de 
factibilidad; Programas de capacitación profe-
sional; Educación para la salud; comercializa-
ción integral para instituciones profesionales y 
empresas del sector; Programas de informática 
medica; Estudios de Bioestadística; Aseso-
ramiento en programas de bioseguridad. c) 
Explotación, alquiler y locación de servicios de 
vehículos de ambulancia, simples o de Unidad 
Coronaria, para servicios urbanos o rurales, 
territorio nacional o provincial, ya sea a través 
de vehículos propios o de terceros, subcontra-
tados; pudiendo establecer sucurs